In [1]:
#Nous voulons prédire une variable binaire - que le salaire soit faible ou élevé.
#Calculez le salaire médian et créez une nouvelle variable binaire qui est vraie lorsque le salaire est élevé
#(au-dessus de la médiane)
#Nous pourrions également effectuer une régression linéaire (ou toute régression) pour prédire la valeur du salaire ici.
#Au lieu de cela, nous allons convertir cela en un problème de classification binaire, 
#en prédisant deux classes, le salaire HAUT vs BAS.

import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/Users/charleshajjar/Desktop/projet_indeed_charles/clean_salary_charles.csv')

In [6]:
#calcule de la médiane qui perméttre de situer la moyenne haute et celle qui est bas 
median_salary = np.median(data.avg)
median_salary

2355.0

In [7]:

data['dumsal'] = (data["avg"] >= data["avg"].median()).astype(int)

In [8]:
data.head()

,Unnamed: 0,Titre,Company,Summaries,avg,cityfrance,city,dumsal
0,0,Alternance - Data Scientist H/F,Malakoff Humanis,Nous recherchons un business data analyst pass...,1125.0,['Paris (75)'],Paris (75),0
1,1,Data Scientist H/F,CCM Benchmark Group,La réalisation d'analyses de data mining répon...,1500.0,['Asnières-sur-Seine (92)'],Asnières-sur-Seine (92),0
2,4,Alternance - Assistant Data Analyst/PMO H/F,Gandi,"Euro Information, filiale Informatique du grou...",1125.0,['Paris (75)'],Paris (75),0
3,5,Développeur confirmé H/F,VONA,Business Developer – Stage de fin d’études – J...,800.0,['Levallois-Perret (92)'],Levallois-Perret (92),0
4,6,Data Analyst·(e) - Stagiaire H/F,Joko,Stagiaire data analyst(e) h/f ou similaire: 1 ...,3150.0,['Paris 9e (75)'],Paris 9e (75),1


In [10]:
data['high_salary'] = [1 if i > median_salary else 0 for i in data.avg]

In [11]:
#dumsal est egame high_salarie 0 veux dire en dessous de la mediane et 1 au dessus de la médiane 
data.head(5)

,Unnamed: 0,Titre,Company,Summaries,avg,cityfrance,city,dumsal,high_salary
0,0,Alternance - Data Scientist H/F,Malakoff Humanis,Nous recherchons un business data analyst pass...,1125.0,['Paris (75)'],Paris (75),0,0
1,1,Data Scientist H/F,CCM Benchmark Group,La réalisation d'analyses de data mining répon...,1500.0,['Asnières-sur-Seine (92)'],Asnières-sur-Seine (92),0,0
2,4,Alternance - Assistant Data Analyst/PMO H/F,Gandi,"Euro Information, filiale Informatique du grou...",1125.0,['Paris (75)'],Paris (75),0,0
3,5,Développeur confirmé H/F,VONA,Business Developer – Stage de fin d’études – J...,800.0,['Levallois-Perret (92)'],Levallois-Perret (92),0,0
4,6,Data Analyst·(e) - Stagiaire H/F,Joko,Stagiaire data analyst(e) h/f ou similaire: 1 ...,3150.0,['Paris 9e (75)'],Paris 9e (75),1,1


In [12]:
data.city.value_counts()[0:30]

France                         87
Bordeaux (33)                  54
Metz (57)                      26
Tassin-la-Demi-Lune (69)       14
Nancy (54)                     11
Levallois-Perret (92)          11
Craponne (69)                  11
Brignais (69)                  10
Paris (75)                      8
Paris 9e (75)                   8
Lozanne (69)                    7
Lyon 9e (69)                    6
Paris 2e (75)                   5
Champagne-au-Mont-d'Or (69)     5
Mérignac (33)                   5
Lyon (69)                       5
Asnières-sur-Seine (92)         3
Gennevilliers (92)              3
Châtillon (92)                  2
Paris 18e (75)                  2
Rueil-Malmaison (92)            2
Île-de-France                   1
Noisy-le-Sec (93)               1
Name: city, dtype: int64

In [14]:
city_dummies = pd.get_dummies(data.city)

X_city = city_dummies
y_city = data.high_salary

In [16]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X_city, y_city, test_size=0.3, random_state=90)

In [17]:
#on predit la les ville qui on un salaire au dessus de la moyenne avec les offre d'emploie receuillie 
rfc = RandomForestClassifier(n_estimators=300, random_state=90)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_pred)
print ("Accuracy Score:", acc.round(3))

s = cross_val_score(rfc, X_city, y_city, cv=10, n_jobs=-1)
print ("Cross Validation Score:\t{:0.3} ± {:0.3}".format(s.mean().round(3), s.std().round(3)))

Accuracy Score: 0.747
Cross Validation Score:	0.72 ± 0.085


In [19]:
#les futures important des ville par rapport au salaire classer.
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_city.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_medians = []
for i in X_city.columns:
    feature_medians.append(np.median(data[data.city == i].avg))

feature_importances['median_salary'] = feature_medians
feature_importances['over_or_under'] = [1 if i > median_salary else 0 for i in feature_importances.median_salary]

feature_importances.sort_values('importance', ascending=False).head(15)



,feature,importance,median_salary,over_or_under
8,Levallois-Perret (92),0.205978,2950.0,1
13,Mérignac (33),0.092120,2100.0,0
7,Gennevilliers (92),0.090976,3736.0,1
12,Metz (57),0.079207,2355.0,0
19,Paris 9e (75),0.064062,2900.0,1
2,Brignais (69),0.053474,2277.5,0
5,Craponne (69),0.048687,2355.0,0
6,France,0.046138,2355.0,0
1,Bordeaux (33),0.044467,2355.0,0
4,Châtillon (92),0.043968,2430.5,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer